# Mushroom Dataset

In [359]:
import tensorflow as tf
from tensorflow import keras

# Helper libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.2.0


Importiamo il dataset dalla [repository GitHub](https://github.com/rirolli/Mushroom/blob/master/mushroom_data_all.csv):

In [360]:
url = 'https://raw.githubusercontent.com/rirolli/Mushroom/master/mushroom_data_all.csv'
mushroom_data = pd.read_csv(url)
print(mushroom_data.columns)

Index(['class_edible', 'cap-shape', 'cap-surface', 'cap-color', 'bruises',
       'odor', 'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
       'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
       'stalk-surface-below-ring', 'stalk-color-above-ring',
       'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number',
       'ring-type', 'spore-print-color', 'population', 'habitat'],
      dtype='object')


Preparazione dei dati rendendoli tutti in forma numerica:

In [0]:
# Le celle con il simbolo '?' indicano dei dati mancanti
# perciò vengono eliminati dal dataset.
mushroom_data.replace('?', np.nan, inplace=True)
mushroom_data.dropna(inplace=True)

# La 'class_edible' indica se un fungo è commestibile ('e') o velenoso ('p').
# Vengono sostituiti i valori 0 per indicare 'velenoso' e 1 per 'commestibile'.
mushroom_data['class_edible'].replace('p', 0, inplace=True)
mushroom_data['class_edible'].replace('e', 1, inplace=True)

# Vengono rimossi tutti gli altri altri caratteri non numerici tramite
# l'ausilio della funzione pd.get_dummies.
cols_to_transform = mushroom_data.columns[1:]
mushroom_data = pd.get_dummies(mushroom_data, columns=cols_to_transform)

Si esegue lo Split del dataset per ricavare i dati di test e di valutazione: 

In [0]:
y = mushroom_data.class_edible
X = mushroom_data.drop(labels=['class_edible'],axis=1)
num_inputs = X.shape[1]

train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=0)

Il modello utilizzato è formato da 3 layer e viene utilizzato come algoritmo di ottimizzazione 'adam':

In [363]:
model = keras.Sequential([
  tf.keras.layers.Dense(12, input_dim=num_inputs, activation='relu'),
  tf.keras.layers.Dense(22, activation='relu'),
  tf.keras.layers.Dense(1)
])

model.summary()

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

Model: "sequential_38"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_96 (Dense)             (None, 12)                1188      
_________________________________________________________________
dense_97 (Dense)             (None, 22)                286       
_________________________________________________________________
dense_98 (Dense)             (None, 1)                 23        
Total params: 1,497
Trainable params: 1,497
Non-trainable params: 0
_________________________________________________________________


Ora è il momento di addestrare il modello tramite i dati di train:

In [0]:
model.fit(train_X, train_y, epochs=20)

Epoch 1/20
133/133 [==============================] - 0s 1ms/step - loss: 0.4153 - accuracy: 0.8604
Epoch 2/20
133/133 [==============================] - 0s 1ms/step - loss: 0.0480 - accuracy: 0.9851
Epoch 3/20
133/133 [==============================] - 0s 1ms/step - loss: 0.0101 - accuracy: 0.9979
Epoch 4/20
133/133 [==============================] - 0s 1ms/step - loss: 0.0022 - accuracy: 0.9998
Epoch 5/20
133/133 [==============================] - 0s 1ms/step - loss: 6.9534e-04 - accuracy: 1.0000
Epoch 6/20
 40/133 [========>.....................] - ETA: 0s - loss: 2.6298e-04 - accuracy: 1.0000

Eseguiamo la valutazione del modello:

In [0]:
# get predicted prices on validation data
test_loss, test_acc = model.evaluate(val_X, val_y)
print(f'\nTest accuracy: {test_acc}')

Ora è il momento di predire i valori di val_X tramite l'addestramento appena effettuato:

In [0]:
predictions = model.predict(val_X)

Effettuiamo una breve stampa dei valori ottenuti tramite `predict` e degli effettivi valori di `val_y`:

In [0]:
print(predictions)
print('-'*10,'\n')
print(val_y)

Tramite i valori trovati generiamo una `confuion_matrix`:

In [0]:
print(confusion_matrix(val_y, predictions.round(), normalize=None))